In [1]:
def get_data(file_path):
    questions, answers = [], []
    with open(file_path, 'r') as f:
        for line in f:
            idx = line.find('_')
            questions.append(line[:idx].strip())
            answers.append(line[idx:-1].strip())
    return questions, answers

# 더하기 문제 데이터를 가져옵니다.
# 어떤 데이터인지 직접 확인해보세요.
file_path = './data/addition.txt'
X_data, y_data = get_data(file_path)

In [2]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 각 문자를 토큰으로 나눠주고 패딩 처리를 합니다.
tokenizer = Tokenizer(char_level=True, filters='')
tokenizer.fit_on_texts(X_data)
tokenizer.fit_on_texts(y_data)

X_sequence = tokenizer.texts_to_sequences(X_data)
y_sequence = tokenizer.texts_to_sequences(y_data)

X_padded = pad_sequences(X_sequence)
y_padded = pad_sequences(y_sequence, padding='post')

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_padded)

In [5]:
embedding_size = 16
hidden_size = 256
vocab_size = len(tokenizer.word_index)+1

# 임베딩 레이어: 인코더와 디코더가 공유합니다.
emb_layer = keras.layers.Embedding(vocab_size, embedding_size)

# 인코더 정의
# 1: 인풋 문자열이 임베딩 레이어를 통과합니다.
# 2: 임베딩 레이어를 통과한 문자열 벡터들이 GRU 층을 통과합니다.
# 3: GRU 층은 인풋 문자열들을 하나의 벡터로 변환합니다. 그 벡터를 아웃풋으로 내보냅니다.encoder_input = keras.layers.Input(shape=(None,))
encoder_input = keras.layers.Input(shape=(None,))
x = emb_layer(encoder_input)
encoder_state = keras.layers.GRU(hidden_size)(x)
encoder = keras.Model(encoder_input, encoder_state)

# 디코더 정의
# 1: 디코더에는 두 개의 인풋이 있습니다. 
  # 인풋1: 인코더에서 넘겨준 은닉 벡터를 받습니다.
  # 인풋2: 디코딩 시작을 알리는 문자열(_)과 다른 문자열들을 받습니다.
  #        다른 문자열이란 (추론단계에서는) 이전 타임스탭에서 확률이 높다고 예측한 문자열입니다.
# 2: 임베딩 레이어는 인코더와 공유합니다. (선택사항입니다.)
# 3: GRU 층을 통과한 뒤, 덴스 층에서 다음에 올 문자열의 확률 값을 구합니다.
decoder_state_input = keras.layers.Input(shape=(None,))  
decoder_input = keras.layers.Input(shape=(None,)) 
x = emb_layer(decoder_input)
x, decoder_state = keras.layers.GRU(hidden_size, return_state=True, return_sequences=True)(x, decoder_state_input)
decoder_output = keras.layers.Dense(vocab_size, activation='softmax')(x)
decoder = keras.Model([decoder_state_input, decoder_input], [decoder_state, decoder_output])

# 인코더-디코더 모델
# 인코더와 디코더를 연결합니다.
decoder_state, model_output = decoder([encoder_state, decoder_input])
model = keras.Model([encoder_input, decoder_input], model_output)

In [6]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit([X_train, y_train[:, :-1]], y_train[:, 1:], 
          epochs=10, batch_size=128, 
         validation_data=([X_test, y_test[:, :-1]], y_test[:, 1:]))

Epoch 1/10
293/293 [==============================] - 4s 12ms/step - loss: 1.8218 - accuracy: 0.3354 - val_loss: 1.7184 - val_accuracy: 0.3453
Epoch 2/10
293/293 [==============================] - 3s 10ms/step - loss: 1.6235 - accuracy: 0.3801 - val_loss: 1.4372 - val_accuracy: 0.4653
Epoch 3/10
293/293 [==============================] - 3s 10ms/step - loss: 1.3275 - accuracy: 0.4940 - val_loss: 1.2768 - val_accuracy: 0.5046
Epoch 4/10
293/293 [==============================] - 3s 10ms/step - loss: 1.1957 - accuracy: 0.5468 - val_loss: 1.1469 - val_accuracy: 0.5650
Epoch 5/10
293/293 [==============================] - 3s 9ms/step - loss: 1.0816 - accuracy: 0.5891 - val_loss: 0.9780 - val_accuracy: 0.6293
Epoch 6/10
293/293 [==============================] - 3s 9ms/step - loss: 0.8470 - accuracy: 0.6813 - val_loss: 0.7298 - val_accuracy: 0.7250
Epoch 7/10
293/293 [==============================] - 3s 9ms/step - loss: 0.6725 - accuracy: 0.7442 - val_loss: 0.6208 - val_accuracy: 0.7621
Ep

In [9]:
import numpy as np

# 이제 테스트 세트 전체에 대해 모델이 문제를 잘 맞히는지 보겠습니다.
# 인코더에는 문제를 인풋으로 넣습니다.
# 디코더에는 시작 문자인 '_'를 넣습니다.
# 디코더에서 다음에 올 문자의 확률을 구합니다. 가장 높은 확률의 문자를 선택합니다.
# 다음 GRU 셀은 이전의 은닉 벡터(hidden_state)와 이전 타임스탭에서 선택된 문자가 입력됩니다.
hidden_state = encoder(X_test)
pred = []
start_id = np.array(tokenizer.texts_to_sequences("_"))
next_input = start_id.repeat(len(X_test), axis=0)
for _ in range(5):
    hidden_state, out = decoder([hidden_state, next_input])
    out = out.numpy().argmax(-1)
    pred.append(out)
    next_input = out
    
X = tokenizer.sequences_to_texts(X_test)
y = tokenizer.sequences_to_texts(np.hstack(pred)) # hstack: pred 안에 있는 넘파이를 가로로(horizontally) 이어줍니다.

In [10]:
# 문제(인코더의 입력값), 정답, 그리고 모델이 추론한 값을 출력합니다.
n_show = 10

for i in range(n_show):
    print('문제:', X[i].replace(' ', ''))
    print('정답:', sum([int(n) for n in X[i].replace(' ', '').split('+')]))
    print('제출:', y[i].replace(' ', ''))
    print('-'*5)

문제: 104+822
정답: 926
제출: 927
-----
문제: 790+986
정답: 1776
제출: 1775
-----
문제: 633+61
정답: 694
제출: 693
-----
문제: 91+904
정답: 995
제출: 995
-----
문제: 360+9
정답: 369
제출: 370
-----
문제: 10+626
정답: 636
제출: 637
-----
문제: 722+585
정답: 1307
제출: 1304
-----
문제: 60+615
정답: 675
제출: 677
-----
문제: 841+69
정답: 910
제출: 908
-----
문제: 44+538
정답: 582
제출: 582
-----
